In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# 讀取訓練數據
file_path="../20240312_Deep_Learning/NN_Classification/housepricedata.csv"
dataset_train = pd.read_csv(file_path)

In [ ]:
# 提取特徵和目標
X = dataset_train.drop(columns=['target_column'])
y = dataset_train['target_column']

In [ ]:
# 切分訓練集和驗證集
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 訓練 SVC 模型
svm_model = SVC()
svm_model.fit(X_train, y_train)
svc_train_preds = svm_model.predict(X_train)
svc_valid_preds = svm_model.predict(X_valid)

In [ ]:
# 訓練 SVR 模型
# svm_model = SVR()
# svm_model.fit(X_train, y_train)
# svr_train_preds = svm_model.predict(X_train)
# svr_valid_preds = svm_model.predict(X_valid)

# 訓練 Decision Tree 模型
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_train_preds = dt_model.predict(X_train)
dt_valid_preds = dt_model.predict(X_valid)

In [ ]:
# 訓練 Linear 模型
# linear_model = LinearRegression()
# linear_model.fit(X_train, y_train)
# linear_train_preds = linear_model.predict(X_train)
# linear_valid_preds = linear_model.predict(X_valid)

# 組合預測結果
ensemble_train_features = np.column_stack((svc_train_preds, dt_train_preds))#, linear_train_preds))
ensemble_valid_features = np.column_stack((svc_valid_preds, dt_valid_preds))#, linear_valid_preds))

In [ ]:
# 組合預測結果
# ensemble_train_features = np.column_stack((svr_train_preds, linear_train_preds))
# ensemble_valid_features = np.column_stack((svr_valid_preds, linear_valid_preds))

# 轉換為 PyTorch 的 Tensor
X_train_tensor = torch.FloatTensor(ensemble_train_features)
X_valid_tensor = torch.FloatTensor(ensemble_valid_features)
y_train_tensor = torch.FloatTensor(y_train.values)
y_valid_tensor = torch.FloatTensor(y_valid.values)

In [ ]:
# 定義原來的神經網路模型
class Neural_Network(nn.Module):
    def __init__(self, num_features: int):
        super(Neural_Network, self).__init__()

        layer_1_neurons = 10
        layer_2_neurons = 20
        layer_3_neurons = 30

        self.fc_input = nn.Linear(num_features, layer_1_neurons)
        self.fc_l1 = nn.Linear(layer_1_neurons, layer_2_neurons)
        self.fc_l2 = nn.Linear(layer_2_neurons, layer_3_neurons)
        self.fc_output = nn.Linear(layer_3_neurons, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc_input(x))
        x = self.relu(self.fc_l1(x))
        x = self.relu(self.fc_l2(x))
        output = self.fc_output(x)
        return output

In [ ]:
# 初始化原來的神經網路模型、損失函數和優化器
model = Neural_Network(num_features=ensemble_train_features.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# 訓練原來的神經網路模型
epochs = 30
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    train_preds = model(X_train_tensor)
    train_loss = criterion(train_preds, y_train_tensor.view(-1, 1))
    train_loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        valid_preds = model(X_valid_tensor)
        valid_loss = criterion(valid_preds, y_valid_tensor.view(-1, 1))

    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss.item():.4f}, Valid Loss: {valid_loss.item():.4f}')